# Fase 1: Extracción de Datos de API de MoviesDataset

In [1]:
#Importamos las librerias necesarias
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
from time import sleep

#importamos librerias de selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

# Configuraciones
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

In [2]:
#creamos el diccionario donde alamcenaremos la información
dic_API_Pelis = {'Tipo':[],
                 'Nombre':[],
                 'Año_estreno':[],
                 'Mes_estreno': [],
                 'Id':[],
                 'Genero':[]}

# creamos una lista para almacenar las respuestas de la funcion de llamada a la API
lista_respuestas = []
lista_info_respuestas = []


#creamos las variables relacionadas con el ENDPOINT
url = "https://moviesdatabase.p.rapidapi.com/titles"

# API_key Nuria = '58296df2a9mshb28a81ec4003c84p15333ejsn11ad25c7c1b8'
# API_key Sharon = '2d7083a699mshd0e8039e2063bedp140338jsn9b141653fce1'
# API_key3 Silvia = '039163dca0msh7ecfddf47a57b42p193f39jsn80c27500a5c2'

# headers = {"X-RapidAPI-Key": API_key,"X-RapidAPI-Host": "moviesdatabase.p.rapidapi.com"}

titleType_list = ['movie', 'short']
genere_list = ['Drama', 'Action', 'Comedy']

# querystring = {"titleType":"short","startYear":"2010","endYear":"2024","genre":"Drama"}


In [3]:
# Definimos una fucnción para extraer información de la API y almacenarla en un diccionario

def llamar_API (url,type='movie', genere='Drama', **dict):
    '''
    Esta función realiza una llamada a una API a partir de una url y guarda la información en un diccionario.

    Parametros:
    -----------
    url (str): la dirección url de la API a la que queremos hacer la llamada
    type (str): El tipo de pelícua que queremos buscar. By default = movie
    genere (dtr): El género que ueremos buscar. By default = Drama
    dict (dict): el diccionario dónde se almacenará la información

    Returns:
    -------
    dict (dict): el diccionario donde se almacenará la información
    '''
    #definimos la querystring y la API key
    querystring = {"titleType":type,"startYear":"2010","endYear":"2024","genre":genere}

    API_key = '2d7083a699mshd0e8039e2063bedp140338jsn9b141653fce1'
    # API_key Nuria = '58296df2a9mshb28a81ec4003c84p15333ejsn11ad25c7c1b8'
    # API_key Sharon = '2d7083a699mshd0e8039e2063bedp140338jsn9b141653fce1'
    # API_key3 Silvia = '039163dca0msh7ecfddf47a57b42p193f39jsn80c27500a5c2'
    headers = {"X-RapidAPI-Key": API_key,"X-RapidAPI-Host": "moviesdatabase.p.rapidapi.com"}    

    #iniciamos el loop while para navegar por las distintas páginas
    while url.startswith('https'):

        #realizmos la llamada y almacenamos los códigos de respuesta en la lista de respuestas
        response_pelis = requests.get(url, headers=headers, params=querystring)
        lista_respuestas.append(response_pelis.status_code)
        lista_info_respuestas.append(response_pelis.reason)

        if response_pelis.status_code == 200:

            #convertimos los resulatos a formato json
            js_response_pelis = response_pelis.json()

            #guardamos la inforación de la API dónde tenemos la dirección de la siguiente página
            next = js_response_pelis['next']

            #guardamos la parte donde tenemos la infomación en una variable
            info_pelis = js_response_pelis['results']

            #guardamos toda la información en el diccionario con un for loop    
            for peli in info_pelis:

                dict['Genero'].append(genere)
                dict['Tipo'].append(peli['titleType']['text'])
                dict['Nombre'].append(peli['titleText']['text'])
                dict['Id'].append(peli['id'])
                dict['Año_estreno'].append(peli['releaseYear']['year'])

                #Hacemos un try/expect para las películas que no se han estrenado todavía            
                try:
                    dict['Mes_estreno'].append(peli['releaseDate']['month'])
                except:            
                    dict['Mes_estreno'].append('por estrenar')

            #iniciamos el loop try-except para que cree la variable de la nueva url mientras tengamos una url en ´next´
            try:
                #definimos la base de la nueva url
                url_fixed = "https://moviesdatabase.p.rapidapi.com"
                #definimos la nueva url
                url=url_fixed+next
                #definimos el nuevo querystring
                querystring = {}

            except:
                break

        else:
            print(f'Error {response_pelis.status_code} en la llamada a la API')
            break

    print(f'Fin: Se han añadido l@s {type}s del género {genere} a tu diccionario.')

In [4]:
# Iniciamos el loop for para conseguir información de todos los tipos y géneros
for type in titleType_list:
    for genere in genere_list:
        llamar_API(url,type,genere, **dic_API_Pelis)

Fin: Se han añadido l@s movies del género Drama a tu diccionario.
Fin: Se han añadido l@s movies del género Action a tu diccionario.
Fin: Se han añadido l@s movies del género Comedy a tu diccionario.
Fin: Se han añadido l@s shorts del género Drama a tu diccionario.
Fin: Se han añadido l@s shorts del género Action a tu diccionario.
Fin: Se han añadido l@s shorts del género Comedy a tu diccionario.


In [5]:
#Almacenamos la infomacion del diccionario en distintas listas, cada una corresponde a una key 
lista_tipos = dic_API_Pelis['Tipo']
lista_nombre = dic_API_Pelis['Nombre']
lista_anno = dic_API_Pelis['Año_estreno']
lista_mes = dic_API_Pelis['Mes_estreno']
lista_id = dic_API_Pelis['Id']

In [6]:
#Creamos un zip con las listas que hemos creado para que nos guarde la infoamción de cada una de las pelis en una tupla
zip_pelis = zip(lista_tipos, lista_nombre, lista_anno, lista_mes, lista_id)
lista_tuplas_pelis = list(zip_pelis)
len(lista_tuplas_pelis)

7105

In [7]:
lista_id

['tt10579986',
 'tt10597228',
 'tt11097384',
 'tt11152168',
 'tt11237950',
 'tt11822244',
 'tt12053936',
 'tt12193500',
 'tt12255500',
 'tt12767592',
 'tt12861438',
 'tt13036100',
 'tt13433802',
 'tt13561128',
 'tt13576680',
 'tt13695406',
 'tt13965596',
 'tt14117392',
 'tt14256984',
 'tt14257662',
 'tt14478740',
 'tt14564000',
 'tt14643262',
 'tt14866430',
 'tt14904970',
 'tt14978566',
 'tt15005992',
 'tt15239678',
 'tt15427838',
 'tt15535164',
 'tt15572712',
 'tt1592253',
 'tt2368853',
 'tt3663576',
 'tt4392566',
 'tt4601194',
 'tt5834410',
 'tt6398614',
 'tt9371598',
 'tt15791240',
 'tt9134042',
 'tt15769770',
 'tt16139258',
 'tt9810236',
 'tt15721572',
 'tt16867258',
 'tt7131764',
 'tt8547668',
 'tt16913084',
 'tt17037988',
 'tt17073016',
 'tt17590672',
 'tt17731524',
 'tt18024180',
 'tt18213692',
 'tt18556552',
 'tt19336000',
 'tt19368948',
 'tt19898506',
 'tt20407238',
 'tt20411966',
 'tt20453194',
 'tt21065476',
 'tt22022452',
 'tt22394800',
 'tt23461428',
 'tt23747498',
 'tt241